# Genetic determinants of incident CH

In [1]:
library(data.table)

In [5]:
setwd("/medpop/esp2/mesbah/projects/ch_progression/aric/gwas/rg2022/")

In [10]:
# load prev. CH GWAS loci 
chip_hg38tohg19 <- fread("ch_rg22.hg19.csv", header=T)
str(chip_hg38tohg19)  

dnmt3a_hg38tohg19 <- fread("dnmt3a_rg22.hg19.csv", header=T)
str(dnmt3a_hg38tohg19)

tet2_hg38tohg19 <- fread("tet2_rg22.hg19.csv", header=T)
str(tet2_hg38tohg19)

Classes 'data.table' and 'data.frame':	57 obs. of  51 variables:
 $ varID            : chr  "1:226374920:A:C" "10:103908421:C:CTT" "11:108043157:T:C" "11:108435509:C:A" ...
 $ chr_hg19         : int  1 10 11 11 11 12 14 17 17 17 ...
 $ POS_hg19         : int  226562621 105668179 107913884 108306236 108315606 26589770 101178715 55463857 7571752 7731041 ...
 $ locus            : chr  "1-1" "10-1" "11-1" "11-1" ...
 $ Chr              : int  1 10 11 11 11 12 14 17 17 17 ...
 $ Pos              : int  226374920 103908421 108043157 108435509 108444879 26436837 100712378 57386496 7668434 7827723 ...
 $ Ref              : chr  "A" "C" "T" "C" ...
 $ Alt              : chr  "C" "CTT" "C" "A" ...
 $ Trait            : chr  "CHIP_inclusive" "CHIP_inclusive" "CHIP_inclusive" "CHIP_inclusive" ...
 $ Model            : chr  "ADD-WGR-FIRTH" "ADD-WGR-FIRTH" "ADD-WGR-FIRTH" "ADD-WGR-FIRTH" ...
 $ Effect_UKB       : num  1.115 0.914 0.921 1.12 1.228 ...
 $ LCI_Effect_UKB   : num  1.086 0.887 0.896 1.09

In [33]:
## Load phenotypes
aric_baseline_n_v05 <- fread("../../pheno/aric_baseline_n_v05_N4189.pheno_ch_status_trajectory.23Mar2023.csv", header=T)
aric_baseline_n_v05$dAge <- aric_baseline_n_v05$Age - aric_baseline_n_v05$age_base

In [34]:
# dbGaP linker
aric_dbgap_linker <- fread("../../pheno/phs000280.v7.pht001441.v7.p1.ARIC_Sample.MULTI.txt.gz", 
                           skip = 10, fill = TRUE, header = T)

aric_dbgap_linker <- aric_dbgap_linker[,c(1,2,4,5)]

aric_dbgap_linker <- subset(aric_dbgap_linker, !duplicated(aric_dbgap_linker$SAMPLE_ID))

table(aric_baseline_n_v05$GWAS_ID %in% aric_dbgap_linker$SAMPLE_ID, exclude = NULL)



TRUE 
4189 

In [35]:
## link dbGaP ids with GWASID
aric_baseline_n_v05 <- merge(aric_dbgap_linker, 
                              aric_baseline_n_v05,
                              by.x = "SAMPLE_ID",
                              by.y = "GWAS_ID")

aric_baseline_n_v05$SUBJECT_ID <- as.numeric(aric_baseline_n_v05$SUBJECT_ID)

head(aric_baseline_n_v05)

SAMPLE_ID,dbGaP_Subject_ID,dbGaP_Sample_ID,SUBJECT_ID,ARIC_ID,GWASID_Visit,cram_id,Visit.x,Gender,Age,...,CH_SF,CH_DDR,CH_DNMT3A,CH_TET2,CH_ASXL1,nonHDL_base_std,age_base_sqr,Center,incident_CH_or_growingClones,dAge
<chr>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,...,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<chr>,<int>,<int>
A00004,288368,1786858,131031724,M102321,A00004_V05,H5TH7DSX2-2-IDUDI0072,V05,F,71,...,NA,NA,1,NA,NA,-0.06472577,2601,M,1,20
A00009,227213,1786861,131051876,F101210,A00009_V05,HNTCYDSX2-2-IDUDI0060,V05,F,76,...,0,0,0,0,0,-0.16103907,3249,F_J,0,19
A00012,232161,1786863,131188895,W121907,A00012_V05,HNTGWDSX2-1-IDUDI0006,V05,F,79,...,0,0,0,0,0,-0.16966414,3249,W,0,22
A00013,222837,1387960,131123798,M263966,A00013_V05,HKNWTDSX2-3-IDUDI0063,V05,M,82,...,NA,NA,NA,1,NA,-0.07766338,3844,M,1,20
A00015,228166,1786865,131154692,F101837,A00015_V05,H252LDSX3-1-IDUDI0036,V05,M,82,...,0,0,0,0,0,-0.13947639,3969,F_J,0,19
A00017,224264,1786866,131017525,M132251,A00017_V05,HY2KGDSX2-1-IDUDI0021,V05,M,83,...,0,0,0,0,0,-0.07047582,3721,M,0,22


In [36]:
## PCA data
pca_ea <- fread("../../pca/ARIC_EurAm_chr1_22.aray_snps.eigenvec", header = T)
# pca_ea$IID <- as.character(pca_ea$IID)
pca_aa <- fread("../../pca/ARIC_AfrAm_chr1_22.aray_snps.eigenvec", header = T)
# pca_ea$IID <- as.character(pca_ea$IID)
table(aric_baseline_n_v05$SUBJECT_ID %in% pca_ea$IID)


FALSE  TRUE 
 1497  2692 

In [37]:
# EA: 2692
aric_baseline_n_v05_ea <- merge(aric_baseline_n_v05, pca_ea, 
                                by.x="SUBJECT_ID", by.y="IID")

nrow(aric_baseline_n_v05_ea)

[1] 2692

In [38]:
# EA: 2692
aric_baseline_n_v05_aa <- merge(aric_baseline_n_v05, pca_aa, 
                                by.x="SUBJECT_ID", by.y="IID")

nrow(aric_baseline_n_v05_aa)

[1] 701

In [50]:
names(aric_baseline_n_v05_ea[, c(1,1,2,6,3:5,55,9:21,109, 61,107,108,112:121,85,86,87,89)])
aric_baseline_n_v05_ea_gwas <- aric_baseline_n_v05_ea[, c(1,1,2,6,3:5,55,9:21,109, 61,107,108,112:121,85,86,87,89)]
names(aric_baseline_n_v05_ea_gwas) <- c("FID","IID","GWAS_ID", names(aric_baseline_n_v05_ea_gwas)[4:39])
str(aric_baseline_n_v05_ea_gwas)

[1] "SUBJECT_ID"                   "SUBJECT_ID"                  
 [3] "SAMPLE_ID"                    "GWASID_Visit"                
 [5] "dbGaP_Subject_ID"             "dbGaP_Sample_ID"             
 [7] "ARIC_ID"                      "Visit.y"                     
 [9] "Gender"                       "Age"                         
[11] "CH_baseline"                  "CH_v05"                      
[13] "incident_CH"                  "incident_CHvaf05"            
[15] "incident_CHvaf10"             "incident_DNMT3A"             
[17] "incident_TET2"                "incident_ASXL1"              
[19] "incident_DTA"                 "incident_SF"                 
[21] "incident_DDR"                 "incident_CH_or_growingClones"
[23] "age_base"                     "age_base_sqr"                
[25] "Center"                       "PC1"                         
[27] "PC2"                          "PC3"                         
[29] "PC4"                          "PC5"                         
[31] "PC6"                          "PC7"                         
[33] "PC8"                          "PC9"                         
[35] "PC10"                         "ever_smoke"                  
[37] "v2_vs_other"                  "Sex"                         
[39] "BMI_cat"

Classes 'data.table' and 'data.frame':	2692 obs. of  39 variables:
 $ FID                         : int  131000113 131000147 131000179 131000253 131000354 131000401 131000455 131000596 131000641 131000672 ...
 $ IID                         : int  131000113 131000147 131000179 131000253 131000354 131000401 131000455 131000596 131000641 131000672 ...
 $ GWAS_ID                     : chr  "A15057" "A04826" "A02787" "A07971" ...
 $ GWASID_Visit                : chr  "A15057_V05" "A04826_V05" "A02787_V05" "A07971_V05" ...
 $ dbGaP_Subject_ID            : int  228789 232712 230453 231733 224837 229698 234835 230455 228925 229174 ...
 $ dbGaP_Sample_ID             : int  1336383 1788922 1788029 1389147 1787805 1388504 1787033 1388831 1336156 1334770 ...
 $ ARIC_ID                     : chr  "W293092" "M127886" "W235739" "M288311" ...
 $ Visit.y                     : chr  "V02" "V02" "V02" "V02" ...
 $ Gender                      : chr  "M" "M" "F" "M" ...
 $ Age                         : int 

In [51]:
names(aric_baseline_n_v05_aa[, c(1,1,2,6,3:5,55,9:21,109, 61,107,108,112:121,85,86,87,89)])
aric_baseline_n_v05_aa_gwas <- aric_baseline_n_v05_aa[, c(1,1,2,6,3:5,55,9:21,109, 61,107,108,112:121,85,86,87,89)]
names(aric_baseline_n_v05_aa_gwas) <- c("FID","IID","GWAS_ID", names(aric_baseline_n_v05_aa_gwas)[4:39])
str(aric_baseline_n_v05_aa_gwas)

[1] "SUBJECT_ID"                   "SUBJECT_ID"                  
 [3] "SAMPLE_ID"                    "GWASID_Visit"                
 [5] "dbGaP_Subject_ID"             "dbGaP_Sample_ID"             
 [7] "ARIC_ID"                      "Visit.y"                     
 [9] "Gender"                       "Age"                         
[11] "CH_baseline"                  "CH_v05"                      
[13] "incident_CH"                  "incident_CHvaf05"            
[15] "incident_CHvaf10"             "incident_DNMT3A"             
[17] "incident_TET2"                "incident_ASXL1"              
[19] "incident_DTA"                 "incident_SF"                 
[21] "incident_DDR"                 "incident_CH_or_growingClones"
[23] "age_base"                     "age_base_sqr"                
[25] "Center"                       "PC1"                         
[27] "PC2"                          "PC3"                         
[29] "PC4"                          "PC5"                         
[31] "PC6"                          "PC7"                         
[33] "PC8"                          "PC9"                         
[35] "PC10"                         "ever_smoke"                  
[37] "v2_vs_other"                  "Sex"                         
[39] "BMI_cat"

Classes 'data.table' and 'data.frame':	701 obs. of  39 variables:
 $ FID                         : int  131000053 131000094 131000655 131001064 131001173 131001245 131001306 131001309 131001392 131001460 ...
 $ IID                         : int  131000053 131000094 131000655 131001064 131001173 131001245 131001306 131001309 131001392 131001460 ...
 $ GWAS_ID                     : chr  "A05162" "A14961" "A12667" "A11283" ...
 $ GWASID_Visit                : chr  "A05162_V05" "A14961_V05" "A12667_V05" "A11283_V05" ...
 $ dbGaP_Subject_ID            : int  230296 229859 224585 228563 229539 224645 226634 232824 229013 231474 ...
 $ dbGaP_Sample_ID             : int  699030 701005 1792295 1389627 699670 699409 699581 701049 699835 698355 ...
 $ ARIC_ID                     : chr  "J187560" "J124491" "J307167" "J304595" ...
 $ Visit.y                     : chr  "V02" "V02" "V01" "V02" ...
 $ Gender                      : chr  "M" "F" "F" "M" ...
 $ Age                         : int  72 69 67

In [52]:
## GWAS_ID==SAMPLE_ID
# fwrite(aric_baseline_n_v05_ea, "../aric_baseline_n_v05_ea_PCA.2023Apr3.csv", 
  #      row.names = F, col.names = T, sep=",", na="NA")
# fwrite(aric_baseline_n_v05_ea_gwas, "../aric_baseline_n_v05_ea_PCA_GWAS.2023Apr3.tsv",
  #      row.names = F, col.names = T, sep="\t", na="NA", quote = F)
# 
# fwrite(aric_baseline_n_v05_aa, "../aric_baseline_n_v05_aa_PCA.2023Apr3.csv", 
  #      row.names = F, col.names = T, sep=",", na="NA")
# fwrite(aric_baseline_n_v05_aa_gwas, "../aric_baseline_n_v05_aa_PCA_GWAS.2023Apr3.tsv",
  #      row.names = F, col.names = T, sep="\t", na="NA", quote = F)


In [2]:
### Prepare data for gwas for growthrate
aric_baseline_n_v05_aa_gwas <- fread("/medpop/esp2/mesbah/projects/ch_progression/aric/gwas/aric_baseline_n_v05_aa_PCA_GWAS.2023Apr3.v2.tsv", 
                                     header=T)
head(aric_baseline_n_v05_aa_gwas)
aric_baseline_n_v05_ea_gwas <- fread("/medpop/esp2/mesbah/projects/ch_progression/aric/gwas/aric_baseline_n_v05_ea_PCA_GWAS.2023Apr3.v2.tsv", 
                                     header=T)
head(aric_baseline_n_v05_ea_gwas)

FID,IID,GWAS_ID,GWASID_Visit,dbGaP_Subject_ID,dbGaP_Sample_ID,ARIC_ID,Visit.y,Gender,Age,...,PC4,PC5,PC6,PC7,PC8,PC9,PC10,ever_smoke,v2_vs_other,Sex
<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>
131000053,131000053,A05162,A05162_V05,230296,699030,J187560,V02,M,72,...,-0.0160964,-0.02318640,-0.00253648,-0.00182851,-0.03706380,-0.00436290,0.020383400,1,V02,M
131000094,131000094,A14961,A14961_V05,229859,701005,J124491,V02,F,69,...,-0.0203355,0.00267189,-0.01572940,-0.00156514,0.01031670,0.04677960,-0.003931650,0,V02,F
131000655,131000655,A12667,A12667_V05,224585,1792295,J307167,V01,F,67,...,-0.0132270,0.00583279,-0.00779501,-0.01494340,-0.00611278,0.01740740,0.002618600,0,Others,F
131001064,131001064,A11283,A11283_V05,228563,1389627,J304595,V02,M,70,...,-0.0166595,-0.02797620,-0.01208170,0.00343874,-0.01595520,0.01301230,-0.028101800,0,V02,M
131001173,131001173,A08339,A08339_V05,229539,699670,J214024,V02,F,84,...,0.0164780,-0.00476401,0.02233330,-0.01398200,0.00148152,-0.02277820,-0.000707081,1,V02,F
131001245,131001245,A07099,A07099_V05,224645,699409,J291749,V02,F,69,...,0.0132752,-0.01870220,-0.02439060,-0.00570036,0.01178660,0.00763499,0.007431030,0,V02,F


FID,IID,GWAS_ID,GWASID_Visit,dbGaP_Subject_ID,dbGaP_Sample_ID,ARIC_ID,Visit.y,Gender,Age,...,PC4,PC5,PC6,PC7,PC8,PC9,PC10,ever_smoke,v2_vs_other,Sex
<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>
131000113,131000113,A15057,A15057_V05,228789,1336383,W293092,V02,M,74,...,0.01125710,-0.007130800,-0.006006790,-2.40823e-05,0.007327390,0.00636429,-0.00978289,0,V02,M
131000147,131000147,A04826,A04826_V05,232712,1788922,M127886,V02,M,80,...,0.00337632,-0.000636593,0.005956820,3.17560e-03,0.005646950,0.00582919,-0.00585952,1,V02,M
131000179,131000179,A02787,A02787_V05,230453,1788029,W235739,V02,F,74,...,-0.00218562,0.000839117,0.000278637,4.24331e-03,-0.005499530,0.01091180,0.01042130,1,V02,F
131000253,131000253,A07971,A07971_V05,231733,1389147,M288311,V02,M,67,...,0.03580850,-0.006886160,-0.003110440,-2.39232e-03,-0.017421700,0.00687928,-0.00980581,1,V02,M
131000354,131000354,A02264,A02264_V05,224837,1787805,F300112,V02,M,81,...,-0.00323656,-0.000967097,0.003248860,1.43675e-03,-0.000950486,0.00335151,0.00647004,1,V02,M
131000401,131000401,A03618,A03618_V05,229698,1388504,M229316,V02,F,77,...,-0.00509369,-0.024892200,0.001788970,6.51412e-03,-0.000148752,0.00869312,-0.00103349,0,V02,F


In [4]:
###
cln_grt.vaf2.DP20_base.corrected_ordered <- fread("/medpop/esp2/mesbah/projects/ch_progression/aric/pheno/cln_grt.vaf2.DP20_base.corrected_ordered.01Dec2023.csv", 
                                                  header = T)

## 
cln_grt.vaf2.DP20_base_allAD5FRRR2.corrected_ordered <- fread("/medpop/esp2/mesbah/projects/ch_progression/aric/pheno/cln_grt.vaf2.DP20_base_allAD5FRRR2.corrected_ordered.01Dec2023.csv", 
                                                              header=T)


In [5]:
names(cln_grt.vaf2.DP20_base.corrected_ordered)

names(cln_grt.vaf2.DP20_base_allAD5FRRR2.corrected_ordered)

[1] "CHROM_POS_GWAS_VISIT_base" "ARIC_ID"                  
  [3] "varID_GWASID"              "GWAS_ID.x"                
  [5] "CHR"                       "POS"                      
  [7] "REF"                       "ALT"                      
  [9] "Gene"                      "AD.v5"                    
 [11] "VAF.v5"                    "DP.v5"                    
 [13] "AD.Alt.v5"                 "FR.Alt.v5"                
 [15] "RR.Alt.v5"                 "Accession"                
 [17] "transcriptOI"              "NonsynOI"                 
 [19] "Func.refGene"              "ExonicFunc.refGene"       
 [21] "AAChange.refGene"          "AD.v2"                    
 [23] "VAF.v2"                    "DP.v2"                    
 [25] "AD.Alt.v2"                 "FR.Alt.v2"                
 [27] "RR.Alt.v2"                 "Batch"                    
 [29] "dAge"                      "dVAF"                     
 [31] "pctVAF"                    "growth_rate"              
 [33] "log_growth_rate"           "Gene_Group"               
 [35] "Gene_Group_2"              "Clone_status"             
 [37] "Trajectory"                "GWAS_ID.y"                
 [39] "GWASID_Visit"              "cram_id"                  
 [41] "Visit.x"                   "Gender"                   
 [43] "Age"                       "CH_baseline"              
 [45] "CH_v05"                    "incident_CH"              
 [47] "incident_CHvaf05"          "incident_CHvaf10"         
 [49] "incident_DNMT3A"           "incident_TET2"            
 [51] "incident_ASXL1"            "incident_DTA"             
 [53] "incident_SF"               "incident_DDR"             
 [55] "death.x"                   "death_date.x"             
 [57] "hg38_CRAM_ID"              "aricid"                   
 [59] "Visit.y"                   "sex"                      
 [61] "race"                      "center"                   
 [63] "dob"                       "date_base"                
 [65] "age_base"                  "fast_8_base"              
 [67] "bmi_base"                  "dm_126_base"              
 [69] "gluc_base"                 "htn_5_base"               
 [71] "chol_med_base"             "statin_base"              
 [73] "cig_base"                  "chd_date"                 
 [75] "is_date"                   "hs_date"                  
 [77] "hf_date"                   "death.y"                  
 [79] "death_date.y"              "lfu_date"                 
 [81] "chd_base"                  "is_base"                  
 [83] "hs_base"                   "hf_base"                  
 [85] "ever_smoke"                "v2_vs_other"              
 [87] "Sex"                       "race_BW"                  
 [89] "BMI_cat"                   "nonHDL_std"               
 [91] "hdl_base_std"              "ldl_base_std"             
 [93] "chol_base_std"             "tg_base_std"              
 [95] "gluc_base_std"             "bmi_base_std"             
 [97] "CHvaf10_baseline"          "CH_baseline_v05"          
 [99] "CHvaf10_baseline_v05"      "CH_DTA"                   
[101] "CH_SF"                     "CH_DDR"                   
[103] "CH_DNMT3A"                 "CH_TET2"                  
[105] "CH_ASXL1"                  "nonHDL_base_std"          
[107] "age_base_sqr"              "Center"                   
[109] "sbp_base"                  "dbp_base"                 
[111] "hem_can_base"              "hem_can_inc_base"         
[113] "hem_can_int_base"          "hem_can_inc_v5"           
[115] "hem_can_int_v5"            "chol_base"                
[117] "hdl_base"                  "ldl_base"                 
[119] "tg_base"                   "chd_is_base"              
[121] "chol_base_INT"             "ldl_base_INT"             
[123] "hdl_base_INT"              "tg_base_INT"              
[125] "nonHDL_base"               "nonHDL_base_INT"          
[127] "bmi_base_INT"              "tg_to_hdl_base"           
[129] "tg_to_hdl_base_INT"        

[1] "CHROM_POS_GWAS_VISIT_base" "ARIC_ID"                  
  [3] "varID_GWASID"              "GWAS_ID.x"                
  [5] "CHR"                       "POS"                      
  [7] "REF"                       "ALT"                      
  [9] "Gene"                      "AD.v5"                    
 [11] "VAF.v5"                    "DP.v5"                    
 [13] "AD.Alt.v5"                 "FR.Alt.v5"                
 [15] "RR.Alt.v5"                 "Accession"                
 [17] "transcriptOI"              "NonsynOI"                 
 [19] "Func.refGene"              "ExonicFunc.refGene"       
 [21] "AAChange.refGene"          "AD.v2"                    
 [23] "VAF.v2"                    "DP.v2"                    
 [25] "AD.Alt.v2"                 "FR.Alt.v2"                
 [27] "RR.Alt.v2"                 "Batch"                    
 [29] "dAge"                      "dVAF"                     
 [31] "pctVAF"                    "growth_rate"              
 [33] "log_growth_rate"           "Gene_Group"               
 [35] "Gene_Group_2"              "Clone_status"             
 [37] "Trajectory"                "GWAS_ID.y"                
 [39] "GWASID_Visit"              "cram_id"                  
 [41] "Visit.x"                   "Gender"                   
 [43] "Age"                       "CH_baseline"              
 [45] "CH_v05"                    "incident_CH"              
 [47] "incident_CHvaf05"          "incident_CHvaf10"         
 [49] "incident_DNMT3A"           "incident_TET2"            
 [51] "incident_ASXL1"            "incident_DTA"             
 [53] "incident_SF"               "incident_DDR"             
 [55] "death.x"                   "death_date.x"             
 [57] "hg38_CRAM_ID"              "aricid"                   
 [59] "Visit.y"                   "sex"                      
 [61] "race"                      "center"                   
 [63] "dob"                       "date_base"                
 [65] "age_base"                  "fast_8_base"              
 [67] "bmi_base"                  "dm_126_base"              
 [69] "gluc_base"                 "htn_5_base"               
 [71] "chol_med_base"             "statin_base"              
 [73] "cig_base"                  "chd_date"                 
 [75] "is_date"                   "hs_date"                  
 [77] "hf_date"                   "death.y"                  
 [79] "death_date.y"              "lfu_date"                 
 [81] "chd_base"                  "is_base"                  
 [83] "hs_base"                   "hf_base"                  
 [85] "ever_smoke"                "v2_vs_other"              
 [87] "Sex"                       "race_BW"                  
 [89] "BMI_cat"                   "nonHDL_std"               
 [91] "hdl_base_std"              "ldl_base_std"             
 [93] "chol_base_std"             "tg_base_std"              
 [95] "gluc_base_std"             "bmi_base_std"             
 [97] "CHvaf10_baseline"          "CH_baseline_v05"          
 [99] "CHvaf10_baseline_v05"      "CH_DTA"                   
[101] "CH_SF"                     "CH_DDR"                   
[103] "CH_DNMT3A"                 "CH_TET2"                  
[105] "CH_ASXL1"                  "nonHDL_base_std"          
[107] "age_base_sqr"              "Center"                   
[109] "sbp_base"                  "dbp_base"                 
[111] "hem_can_base"              "hem_can_inc_base"         
[113] "hem_can_int_base"          "hem_can_inc_v5"           
[115] "hem_can_int_v5"            "chol_base"                
[117] "hdl_base"                  "ldl_base"                 
[119] "tg_base"                   "chd_is_base"              
[121] "chol_base_INT"             "ldl_base_INT"             
[123] "hdl_base_INT"              "tg_base_INT"              
[125] "nonHDL_base"               "nonHDL_base_INT"          
[127] "bmi_base_INT"              "tg_to_hdl_base"           
[129] "tg_to_hdl_base_INT"        

In [6]:
table(cln_grt.vaf2.DP20_base.corrected_ordered$ARIC_ID %in% aric_baseline_n_v05_aa_gwas$ARIC_ID)
table(cln_grt.vaf2.DP20_base.corrected_ordered$ARIC_ID %in% aric_baseline_n_v05_ea_gwas$ARIC_ID)

table(cln_grt.vaf2.DP20_base_allAD5FRRR2.corrected_ordered$ARIC_ID %in% aric_baseline_n_v05_aa_gwas$ARIC_ID)
table(cln_grt.vaf2.DP20_base_allAD5FRRR2.corrected_ordered$ARIC_ID %in% aric_baseline_n_v05_ea_gwas$ARIC_ID)


FALSE  TRUE 
  797   167 


FALSE  TRUE 
  354   610 


FALSE  TRUE 
  540   118 


FALSE  TRUE 
  241   417 

In [8]:
head(cln_grt.vaf2.DP20_base.corrected_ordered[,c(1:3, 183:189)],4)
head(cln_grt.vaf2.DP20_base_allAD5FRRR2.corrected_ordered[,c(1:3, 183:189)],4)

CHROM_POS_GWAS_VISIT_base,ARIC_ID,varID_GWASID,dVAF_by_dT,logdVAF_by_dT,Variant_type,Gene_Group_R882,Gene_R882,is_notHiSeq,is_notMUTECT
<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>
chr2_25744364_A00651_V02,W321690,chr2_25744364_G_GC_A00651,0.004363158,0.35375925,INDEL,Other,ASXL2,0,1
chr16_30729462_A07730_V02,W320923,chr16_30729462_TC_T_A07730,0.002152632,0.31663985,INDEL,Other,SRCAP,0,1
chr4_105275518_A11085_V02,W320503,chr4_105275518_CT_C_A11085,0.004661905,0.32797869,INDEL,TET2,TET2,0,1
chr9_5073770_A08738_V02,W320457,chr9_5073770_G_T_A08738,0.009952381,0.08071571,Missense,Other,JAK2,1,0


CHROM_POS_GWAS_VISIT_base,ARIC_ID,varID_GWASID,dVAF_by_dT,logdVAF_by_dT,Variant_type,Gene_Group_R882,Gene_R882,is_notHiSeq,is_notMUTECT
<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>
chr2_25744364_A00651_V02,W321690,chr2_25744364_G_GC_A00651,0.004363158,0.35375925,INDEL,Other,ASXL2,0,1
chr4_105275518_A11085_V02,W320503,chr4_105275518_CT_C_A11085,0.004661905,0.32797869,INDEL,TET2,TET2,0,1
chr9_5073770_A08738_V02,W320457,chr9_5073770_G_T_A08738,0.009952381,0.08071571,Missense,Other,JAK2,1,0
chr2_25234311_A16062_V02,W319785,chr2_25234311_CG_C_A16062,0.004105263,0.02967200,INDEL,DNMT3A_nonR882,DNMT3A_nonR882,0,0


In [9]:
grt_aa.dp20 <- merge(aric_baseline_n_v05_aa_gwas, 
                     cln_grt.vaf2.DP20_base.corrected_ordered[,c(1:3, 183:189)], 
                     by="ARIC_ID")

grt_ea.dp20 <- merge(aric_baseline_n_v05_ea_gwas, 
                     cln_grt.vaf2.DP20_base.corrected_ordered[,c(1:3, 183:189)], 
                     by="ARIC_ID")

grt_aa.dp20allAD5FRRR2 <- merge(aric_baseline_n_v05_aa_gwas, 
                     cln_grt.vaf2.DP20_base_allAD5FRRR2.corrected_ordered[,c(1:3, 183:189)], 
                     by="ARIC_ID")

grt_ea.dp20allAD5FRRR2 <- merge(aric_baseline_n_v05_ea_gwas, 
                     cln_grt.vaf2.DP20_base_allAD5FRRR2.corrected_ordered[,c(1:3, 183:189)], 
                     by="ARIC_ID")

In [10]:
head(grt_aa.dp20)
head(grt_ea.dp20)

ARIC_ID,FID,IID,GWAS_ID,GWASID_Visit,dbGaP_Subject_ID,dbGaP_Sample_ID,Visit.y,Gender,Age,...,Sex,CHROM_POS_GWAS_VISIT_base,varID_GWASID,dVAF_by_dT,logdVAF_by_dT,Variant_type,Gene_Group_R882,Gene_R882,is_notHiSeq,is_notMUTECT
<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<int>,...,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>
F108396,131164765,131164765,A01355,A01355_V05,228830,698247,V02,M,71,...,M,chrX_45083470_A01355_V02,chrX_45083470_C_T_A01355,0.002450000,0.28598042,Stopgain,Other,KDM6A,0,1
F120829,131047508,131047508,A11026,A11026_V05,228749,700233,V03,M,77,...,M,chr13_32758127_A11026_V03,chr13_32758127_G_T_A11026,0.001889474,0.30979495,Stopgain,Other,PDS5B,0,1
F123365,131123957,131123957,A05140,A05140_V05,223229,699024,V03,M,69,...,M,chr4_105272637_A05140_V03,chr4_105272637_CT_C_A05140,0.005935294,0.40692386,INDEL,TET2,TET2,0,1
F143923,131026458,131026458,A16098,A16098_V05,227512,701242,V02,M,75,...,M,chr2_25246159_A16098_V02,chr2_25246159_C_T_A16098,0.007809524,0.09801848,Splicing,DNMT3A_nonR882,DNMT3A_nonR882,0,0
F156749,131003516,131003516,A09965,A09965_V05,232484,700011,V03,F,85,...,F,chr17_60663118_A09965_V03,chr17_60663118_C_T_A09965,0.012347059,0.44998192,Stopgain,DDR,PPM1D,0,1
F205527,131149782,131149782,A05890,A05890_V05,229747,699176,V02,M,80,...,M,chr4_105236124_A05890_V02,chr4_105236124_GCACAAACACAACCATCC_G_A05890,0.002450000,0.28598042,INDEL,TET2,TET2,0,1


ARIC_ID,FID,IID,GWAS_ID,GWASID_Visit,dbGaP_Subject_ID,dbGaP_Sample_ID,Visit.y,Gender,Age,...,Sex,CHROM_POS_GWAS_VISIT_base,varID_GWASID,dVAF_by_dT,logdVAF_by_dT,Variant_type,Gene_Group_R882,Gene_R882,is_notHiSeq,is_notMUTECT
<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<int>,...,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>
F100601,131179968,131179968,A09377,A09377_V05,229137,1790886,V03,F,83,...,F,chr2_25248155_A09377_V03,chr2_25248155_GCAGGAGGGCTGGCCTCCTCCACCTT_G_A09377,0.009573684,0.39508378,INDEL,DNMT3A_nonR882,DNMT3A_nonR882,0,1
F100920,131156438,131156438,A14695,A14695_V05,235154,1793159,V03,F,83,...,F,chr2_25240714_A14695_V03,chr2_25240714_G_A_A14695,0.006845000,0.36112830,Missense,DNMT3A_nonR882,DNMT3A_nonR882,0,1
F101063,131024996,131024996,A14432,A14432_V05,227292,1336307,MRI,M,81,...,M,chrX_155120076_A14432_MRI,chrX_155120076_C_T_A14432,0.107128571,1.27466547,Stopgain,Other,BRCC3,0,1
F101108,131088751,131088751,A01650,A01650_V05,230347,1334735,V02,M,83,...,M,chr17_76736877_A01650_V02,chr17_76736877_G_T_A01650,0.002359091,0.28426495,Missense,SF,SRSF2,0,1
F101279,131010885,131010885,A11069,A11069_V05,224927,1335914,V02,M,70,...,M,chr2_25240420_A11069_V02,chr2_25240420_T_C_A11069,0.001636364,0.03215145,Missense,DNMT3A_nonR882,DNMT3A_nonR882,1,0
F102435,131177079,131177079,A16447,A16447_V05,234825,1336548,V03,F,73,...,F,chr2_25244340_A16447_V03,chr2_25244340_T_C_A16447,0.002195000,0.30433874,Splicing,DNMT3A_nonR882,DNMT3A_nonR882,0,1


In [11]:
ls()

[1] "aric_baseline_n_v05_aa_gwas"                         
[2] "aric_baseline_n_v05_ea_gwas"                         
[3] "cln_grt.vaf2.DP20_base.corrected_ordered"            
[4] "cln_grt.vaf2.DP20_base_allAD5FRRR2.corrected_ordered"
[5] "grt_aa.dp20"                                         
[6] "grt_aa.dp20allAD5FRRR2"                              
[7] "grt_ea.dp20"                                         
[8] "grt_ea.dp20allAD5FRRR2"

In [12]:
## GWAS_ID==SAMPLE_ID
# fwrite(grt_aa.dp20[,c(2,3,1,4:47)], "/medpop/esp2/mesbah/projects/ch_progression/aric/gwas/grt_aa.dp20.1Dec2023.csv", 
  #      row.names = F, col.names = T, sep=",", na="NA")

# fwrite(grt_aa.dp20[,c(2,3,1,4:47)], "/medpop/esp2/mesbah/projects/ch_progression/aric/gwas/grt_aa.dp20.1Dec2023.tsv", 
  #      row.names = F, col.names = T, sep="\t", na="NA", quote = F)

# fwrite(grt_ea.dp20[,c(2,3,1,4:47)], "/medpop/esp2/mesbah/projects/ch_progression/aric/gwas/grt_ea.dp20.1Dec2023.csv", 
  #      row.names = F, col.names = T, sep=",", na="NA")

# fwrite(grt_ea.dp20[,c(2,3,1,4:47)], "/medpop/esp2/mesbah/projects/ch_progression/aric/gwas/grt_ea.dp20.1Dec2023.tsv", 
  #      row.names = F, col.names = T, sep="\t", na="NA", quote = F)

# grt_aa.dp20allAD5FRRR2
# fwrite(grt_aa.dp20allAD5FRRR2[,c(2,3,1,4:47)], "/medpop/esp2/mesbah/projects/ch_progression/aric/gwas/grt_aa.dp20allAD5FRRR2.1Dec2023.csv", 
  #      row.names = F, col.names = T, sep=",", na="NA")

# fwrite(grt_aa.dp20allAD5FRRR2[,c(2,3,1,4:47)], "/medpop/esp2/mesbah/projects/ch_progression/aric/gwas/grt_aa.dp20allAD5FRRR2.1Dec2023.tsv", 
  #      row.names = F, col.names = T, sep="\t", na="NA", quote = F)

# fwrite(grt_ea.dp20allAD5FRRR2[,c(2,3,1,4:47)], "/medpop/esp2/mesbah/projects/ch_progression/aric/gwas/grt_ea.dp20allAD5FRRR2.1Dec2023.csv", 
  #      row.names = F, col.names = T, sep=",", na="NA")

# fwrite(grt_ea.dp20allAD5FRRR2[,c(2,3,1,4:47)], "/medpop/esp2/mesbah/projects/ch_progression/aric/gwas/grt_ea.dp20allAD5FRRR2.1Dec2023.tsv", 
 #       row.names = F, col.names = T, sep="\t", na="NA", quote = F)

In [13]:
names(grt_ea.dp20[,c(2,3,1,4:47)])

[1] "FID"                          "IID"                         
 [3] "ARIC_ID"                      "GWAS_ID"                     
 [5] "GWASID_Visit"                 "dbGaP_Subject_ID"            
 [7] "dbGaP_Sample_ID"              "Visit.y"                     
 [9] "Gender"                       "Age"                         
[11] "CH_baseline"                  "CH_v05"                      
[13] "incident_CH"                  "incident_CHvaf05"            
[15] "incident_CHvaf10"             "incident_DNMT3A"             
[17] "incident_TET2"                "incident_ASXL1"              
[19] "incident_DTA"                 "incident_SF"                 
[21] "incident_DDR"                 "incident_CH_or_growingClones"
[23] "age_base"                     "age_base_sqr"                
[25] "Center"                       "PC1"                         
[27] "PC2"                          "PC3"                         
[29] "PC4"                          "PC5"                         
[31] "PC6"                          "PC7"                         
[33] "PC8"                          "PC9"                         
[35] "PC10"                         "ever_smoke"                  
[37] "v2_vs_other"                  "Sex"                         
[39] "CHROM_POS_GWAS_VISIT_base"    "varID_GWASID"                
[41] "dVAF_by_dT"                   "logdVAF_by_dT"               
[43] "Variant_type"                 "Gene_Group_R882"             
[45] "Gene_R882"                    "is_notHiSeq"                 
[47] "is_notMUTECT"